### PII (Personally Identifiable Information) Detection with a pre-trained BOLT NER model

In this notebook, we will show how to use ThirdAI's pre-trained PII detection model on your dataset. This model was trained on a proprietaty synthetic dataset generated from GPT-4. This is a multi-lingual model that was trained on English, French, Spanish and Italian data. It detects the following types of PII:

* NAME
* EMAIL
* HOMEADDRESS
* DOB
* UIN
* PHONENUMBER
* DATE
* VEHICLEUIN
* AGE
* GENDER
* HEIGHT
* USERNAME
* PASSWORD
* OCCUPATION
* ACCOUNTNUMBER
* PIN
* CREDITCARDNUMBER
* SEX
* AMOUNT
* ACCOUNTNAME
* IBAN
* ORGANIZATION
* URL
* CREDITCARDCVV

The latter part of the script shows how to load a pretrained model and train it on new PII entities.

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install datasets

### Activate your ThirdAI License Key

You can apply for a trial license [here](https://www.thirdai.com/try-bolt/) .

In [ ]:
import os
from thirdai import bolt, licensing
import utils

import os

if "THIRDAI_KEY" in os.environ:
    licensing.activate(os.environ["THIRDAI_KEY"])
else:
    licensing.activate("")  # Enter your ThirdAI key here

### Download the Model

In [ ]:
import os

if not os.path.isdir("./models/"):
    os.system("mkdir ./models/")

if not os.path.exists("./models/pretrained_multilingual.model"):
    os.system(
        "wget -nv -O ./models/pretrained_multilingual.model 'https://www.dropbox.com/scl/fi/jxo7pk93wubae5zxatx5m/udt_pretrained_small.bolt?rlkey=cq1fz98z33ljem6kb4i5o5oki&st=98aut7bu&dl=0'"
    )

### Load the Model

In [ ]:
pii_model = bolt.UniversalDeepTransformer.load("./models/pretrained_multilingual.model")

### Use Pretrained Model Out of the Box

In [ ]:
sample_sentence = "I'm Robert. I work for as an AI Engineer for a startup in Houston. I want to apply for a credit card. My email is robbie@gmail.com."

predicted_tags = pii_model.predict({"source": sample_sentence}, top_k=1)

for i, token in enumerate(sample_sentence.split()):
    if predicted_tags[i][0][0] != "O":
        print(token + " : " + predicted_tags[i][0][0])

In [ ]:
sample_sentence = "I'm Siddharth. I work at for a big multinational company in Mountain View. I want to cancel my credit card with number 4147202361663155."

predicted_tags = pii_model.predict({"source": sample_sentence}, top_k=1)

for i, token in enumerate(sample_sentence.split()):
    if predicted_tags[i][0][0] != "O":
        print(token + " : " + predicted_tags[i][0][0])

## Finetune a pretrained model on your own data

#### Create a Tag to label Map

Tag to Label Map is used to map text entities to their corresponding integer labels while training/inferencing using a model. 

Note: Ensure that the tags in your dataset should not be outside of TAG_MAP

In [ ]:
TAGS= [
    "B-PER",
    "I-PER",
    "B-ORG",
    "I-ORG",
    "B-LOC",
    "I-LOC",
    "B-MISC",
    "I-MISC",
]

### Download and Process the dataset

In [ ]:
train_file = utils.download_conll_dataset_as_file("train")
validation_file = utils.download_conll_dataset_as_file("validation")

### Initialize a NER Model

In [ ]:
ner_model = bolt.UniversalDeepTransformer(
    data_types={
        "source": bolt.types.text(), 
        "target": bolt.types.token_tags(tags=TAGS, default_tag="O")
    },
    target="target",
    pretrained_model=pii_model,
)

Call the train function for the NER model and pass the training file to the function [required]. All other parameters are optional.

In [ ]:
ner_model.train(
    train_file,
    epochs=2,
    learning_rate=0.001,
    batch_size=1024,
    metrics=["loss"],
    validation=bolt.Validation(filename=validation_file, metrics=["loss"])
)

### Evaluating Finetuned Model on Test Dataset

In [ ]:
test_data = utils.load_dataset("conll2003")["test"]

predictions = []
actuals = []

labels = ["O"] + TAGS

for example in test_data:
    tokens = {"source": " ".join(example["tokens"])}
    actual_tags = [labels[tag] for tag in example["ner_tags"]]

    # Predict and evaluate
    predicted_tags = ner_model.predict(tokens, top_k=1)

    predictions.extend(predicted_tags)
    actuals.extend(actual_tags)

correct_predictions = sum(p[0][0] == a for p, a in zip(predictions, actuals))
total_predictions = len(predictions)
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")